In [18]:
import os
import pandas
from tabulate import tabulate
#去powershell download

In [19]:
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

In [20]:
class GameHistory():
    #初始化
    def __init__(self, game_dimension=(19, 19), game_players=6, game_members=8):
        self.game_history_folder = 'C:\\Users\\user\\anaconda3\\Projects\\BizCases\\SurvivalGame'
        self.game_dimension = game_dimension
        self.game_players = 6
        self.game_members = 8
        self.game_history = None
    #讀檔
    def get_game_history(self):
        game_history_csv = '%s/GameHistory/GAME-%d-%d.csv' % (
            self.game_history_folder, self.game_players, self.game_members)
        #若不存在
        if not os.path.exists(game_history_csv):
            print('%s 檔案不存在' % game_history_csv)
            self.game_history = None
            return None
        #若存在則讀入game_history
        self.game_history = pandas.read_csv(game_history_csv, header=0, encoding='utf8')
        #讀入後排序
        self.game_history.sort_values(by=['epoch', 'player'], inplace=True)
        return self.game_history
    
    #判斷檔案是否存在
    def game_history_exists(self):
        return self.game_history is not None
    
    #最後一個世代(539)
    def get_game_last_epoch(self):
        _last_epoch = -1 #初值
        
        if self.game_history_exists():
            _last_epoch = self.game_history['epoch'].unique().max()     
        return _last_epoch
    
    #輸入參數,尋找特定世代的整筆資料
    def get_game_epoch(self, my_game_epoch=0):
        return self.game_history.loc[self.game_history['epoch'] == my_game_epoch].copy()
    #尋找指定刪減隊伍數後的世代及國家
    def get_game_survivor_left(self, my_game_survivor_left=1):
        _game_epochs = sorted(self.game_history['epoch'].unique()) #陣列,各個世代
        _game_players=self.game_history['player'].unique() #各個player
        
        for _game_epoch in _game_epochs: #一個世代一個世代找
            _survivor_left_epoch = self.game_history.loc[self.game_history['epoch'] == _game_epoch].copy()
            _survivor_left = _survivor_left_epoch.groupby(['player']).agg('count')
                   
            if _survivor_left.shape[0] == my_game_survivor_left:
                return self.game_history.loc[self.game_history['epoch'] == _game_epoch]
            
        return None
    #loc['epoch']用名稱指定
    #iloc[index]用index指定
    def output_game_history(self, my_game_history):
        if my_game_history.empty:
            return
        
        print(tabulate(my_game_history[['epoch', 'player', 'x', 'y', 'weight']], 
               headers=['世代', '玩家', '座標-X', '座標-Y', '權重'], showindex=False)) #tabulate(依序放入x軸,依序放入y軸)

In [23]:
t=GameHistory()
t.get_game_history()
#t.get_game_last_epoch()

,epoch,player,member,x,y,dir,weight
8,0,俄,0,5,15,-1,1
9,0,俄,1,5,16,-1,1
10,0,俄,2,15,6,-1,1
11,0,俄,3,15,14,-1,1
12,0,俄,4,2,13,-1,1
...,...,...,...,...,...,...,...
3310,537,俄,0,17,4,0,6
3309,537,法,0,1,17,3,2
3312,538,俄,0,11,17,4,6
3311,538,法,0,1,15,1,2


In [22]:
t.get_game_survivor_left(4)

,epoch,player,member,x,y,dir,weight
1057,40,俄,0,0,0,4,6
1058,40,俄,1,18,12,2,1
1059,40,俄,2,15,17,3,1
1061,40,德,0,12,3,1,1
1062,40,德,1,15,14,2,1
1063,40,德,2,1,12,6,1
1064,40,德,4,16,4,7,1
1055,40,法,0,16,11,6,2
1056,40,法,2,10,13,1,2
1060,40,美,1,15,12,4,1


In [10]:
t.output_game_history(t.game_history)

  世代  玩家      座標-X    座標-Y    權重
------  ------  --------  --------  ------
     0  俄             5        15       1
     0  俄             5        16       1
     0  俄            15         6       1
     0  俄            15        14       1
     0  俄             2        13       1
     0  俄            17         1       1
     0  俄             9         7       1
     0  俄             9         4       1
     0  德            15         5       1
     0  德            14         7       1
     0  德             6        10       1
     0  德             7         3       1
     0  德            18        11       1
     0  德            18         4       1
     0  德             2        10       1
     0  德            17         4       1
     0  法             6         8       1
     0  法             2         8       1
     0  法             8        13       1
     0  法             3        13       1
     0  法             4         8       1
     0  法             5        13       1


In [11]:
#子繼承父(GameHistory)
class PlotGames(GameHistory):
    def __init__(self, game_dimension=(19, 19), game_players=6, game_members=8): #父親的初始化條件
        GameHistory.__init__(self, game_dimension, game_players, game_members)
        self.plot_font = font_manager.FontProperties(fname='C:/Windows/Fonts/simsun.ttc', size=10) #繪圖字型大小
    
    #依照存活隊伍數繪圖
    def plot_board_by_survivor_left(self, my_game_survivor_left=1):
        my_game_epoch_history = self.get_game_survivor_left(my_game_survivor_left)
        my_game_players_names = self.game_history['player'].unique()
        my_game_epoch = my_game_epoch_history['epoch'].values[0]

        fig, ax = plt.subplots(dpi=120, figsize=(4, 4))#圖畫紙
        #設定值域
        ax.set_xlim(0, self.game_dimension[0])#長
        ax.set_ylim(0, self.game_dimension[1])#寬
        #設定顯示值為整數
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))
        
        text_style = dict(size=10, color='gray')#繪圖字型大小及顏色

        for player_id in range(len(my_game_players_names)):
            current_game_player = my_game_players_names[player_id]#一個一個跑player的名字
            my_game_epoch_history_player = my_game_epoch_history.loc[my_game_epoch_history['player'] == current_game_player].copy()#個別印出player的整筆資料

            if my_game_epoch_history_player.shape[0] == 0:
                continue
            #設計圖上的點
            ax.scatter(my_game_epoch_history_player['x'],
                       my_game_epoch_history_player['y'],
                       marker=player_id,
                       s=20 * my_game_epoch_history_player['weight'],
                       label=current_game_player)
            
            for i in range(my_game_epoch_history_player.shape[0]):
                _row = my_game_epoch_history_player.iloc[i] #一筆一筆的資料
                _x = _row['x']
                _y = _row['y']
                _s = str(_row['member'])

                ax.text(_x, _y, _s, **text_style)

        ax.legend(loc='best', prop=self.plot_font)
        ax.grid(True)
        
        #匯出png檔
        game_history_png = '%s/GamePlots/GAME-%d-%d-%03d.png' % (
            self.game_history_folder, self.game_players, self.game_members, my_game_epoch)
        
        fig.savefig(game_history_png)
        fig.clf()
        plt.close(fig)
    
    #依照世代繪圖
    def plot_board_by_epoch(self, my_game_epoch=0):
        my_game_epoch_history = self.game_history.loc[self.game_history['epoch'] == my_game_epoch].copy()
        my_game_players_names = self.game_history['player'].unique()

        fig, ax = plt.subplots(dpi=120, figsize=(4, 4))

        ax.set_xlim(0, self.game_dimension[0])
        ax.set_ylim(0, self.game_dimension[1])

        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))

        text_style = dict(size=10, color='gray')

        for player_id in range(len(my_game_players_names)):
            current_game_player = my_game_players_names[player_id]
            my_game_epoch_history_player = my_game_epoch_history.loc[my_game_epoch_history['player'] == current_game_player].copy()

            if my_game_epoch_history_player.shape[0] == 0:
                continue

            ax.scatter(my_game_epoch_history_player['x'],
                       my_game_epoch_history_player['y'],
                       marker=player_id,
                       s=20 * my_game_epoch_history_player['weight'],
                       label=current_game_player)

            for i in range(my_game_epoch_history_player.shape[0]):
                _row = my_game_epoch_history_player.iloc[i]
                _x = _row['x']
                _y = _row['y']
                _s = str(_row['member'])

                ax.text(_x, _y, _s, **text_style)

        ax.legend(loc='best', prop=self.plot_font)
        ax.grid(True)

        game_history_png = '%s/GamePlots/GAME-%d-%d-%03d.png' % (
            self.game_history_folder, self.game_players, self.game_members, epoch)
        
        fig.savefig(game_history_png)
        fig.clf()
        plt.close(fig)

In [12]:
if __name__ == '__main__': #主程式
    worker = PlotGames(game_dimension=(19, 19), 
                       game_players=6, 
                       game_members=8)
    
    game_history = worker.get_game_history()
    
    for player_left in range(1, 6): #1~5
        my_game_history_survivors = worker.get_game_survivor_left(player_left)
        if my_game_history_survivors is None: #如果是空的那就找下一個
            continue
            
        worker.output_game_history(my_game_history_survivors)
        worker.plot_board_by_survivor_left(player_left)
        
    for epoch in range(worker.get_game_last_epoch()+1):
        worker.plot_board_by_epoch(epoch)

  世代  玩家      座標-X    座標-Y    權重
------  ------  --------  --------  ------
   539  俄            17        17       6
  世代  玩家      座標-X    座標-Y    權重
------  ------  --------  --------  ------
   499  俄            17        17       6
   499  法             3         3       2
  世代  玩家      座標-X    座標-Y    權重
------  ------  --------  --------  ------
   257  俄            14        16       6
   257  法            15        16       2
   257  法             8         4       2
  世代  玩家      座標-X    座標-Y    權重
------  ------  --------  --------  ------
   232  俄             1        10       6
   232  俄             9         2       1
   232  法            10         1       2
   232  法            16         4       2
  世代  玩家      座標-X    座標-Y    權重
------  ------  --------  --------  ------
   213  俄            13        10       6
   213  俄            11         0       1
   213  德            12         2       1
   213  法            14        14       2
   213  法            14        1